In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

# Print assigned GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

# Set TensorFlow version to 1.x
%tensorflow_version 1.x

# Change dir to project dir
%cd /content/gdrive/MyDrive/Colab\ Data/COCO-Human-Pose

In [ ]:
# Install colab_ssh on google colab
#!pip install colab_ssh --upgrade

#from colab_ssh import launch_ssh_cloudflared
#launch_ssh_cloudflared(password="")


In [ ]:
# (<5 mins)
# Setup train/val symlinks for this runtime
# puts them in /content/datasets/coco
# ensure you have added a Drive shortcut to the dataset in the root of your Drive
# this is necessary as the symlinks need a destination that exists
# the dataset is hosted here: https://drive.google.com/drive/folders/1EVsLBRwT2njNWOrmBAhDHvvB8qrd9pXT 
import time
from datetime import datetime, timedelta

start = time.time()

!unzip /content/gdrive/MyDrive/COCO/2017/train2017_symlinks-1000.zip -d -q /content
!unzip /content/gdrive/MyDrive/COCO/2017/val2017_symlinks-1000.zip -d -q /content

end = time.time()

setup_time = end - start

print("Total setup time: {}".format(str(timedelta(seconds=setup_time))))

In [ ]:
!ls /content/datasets/coco/train2017 | wc -l
!ls /content/datasets/coco/val2017 | wc -l

In [ ]:
# (10-20 mins)
# Populate the runtimes filesystem cache with dest directories for fast and predictable load times
start = time.time()

import os
def print_files_in_dir(path):
    print(len(os.listdir(path)), " files at ", path)

for i in range(0,6):
    print_files_in_dir('/content/gdrive/MyDrive/COCO/2017/val2017_sub-1000/' + str(i))

for i in range(0,119):
    print_files_in_dir('/content/gdrive/MyDrive/COCO/2017/train2017_sub-1000/' + str(i))
  
end = time.time()

setup_time = end - start

print("Total setup time: {}".format(str(timedelta(seconds=setup_time))))

In [ ]:
# Example Usage of Data Generator
from constants import INPUT_DIM
from constants import OUTPUT_DIM
from data_generator import DataGenerator

train_gen = DataGenerator("data/one_ann_min10kp_imgs.csv",
                          "/content/datasets/coco/",
                          INPUT_DIM,
                          OUTPUT_DIM,
                          num_hg_blocks=8,
                          batch_size=1,
                          shuffle=True,
                          online_fetch=True)
# now let's train the model
#model.fit(train_gen, epochs=5, ...)

In [ ]:
!git submodule update --init --recursive

In [ ]:
!git checkout robert/training-setup



In [ ]:
!python3 train.py --epochs 100 --batch 24 --hourglass 1

In [ ]:
# Test the generator
import time
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 10.0)

from HeatMap import HeatMap # https://github.com/LinShanify/HeatMap

start = time.time()
X_batch, y_batch = train_gen[15]
X,y = X_batch[0], y_batch[0] # take first example of batch
print("Retrieving batch took: ",time.time() - start, " s")
print(X.shape,y.shape)

for i in range(NUM_COCO_KEYPOINTS):
  hm = HeatMap(X,y[:,:,i])
  hm.plot(transparency=0.5,show_axis=True,show_colorbar=True)

In [ ]:
for i in range(17):
  X, y = X_batch[i], y_batch[i]
  xs = [v for i,v in enumerate(y) if i%3==0]
  ys = [v for i,v in enumerate(y) if i%3==1]
  plt.subplot(5,5,i+1)
  plt.imshow(X)
  plt.scatter(xs,ys)